In [38]:
library(bootstrap)
library(boot)
data <- scor

In [88]:
cov_mle <- function(data){
    m <- 0
    for (i in 1:ncol(data)){
        mat <- matrix(data[,i]-mean(data[,i]), nrow = nrow(data))
        transmat  <- t(mat)
        prod  <-  (mat%*%transmat)/nrow(data)
        m <- m + prod
    }
    return (m)
}

In [91]:
theta_hat <- function(data,i){
    A <- cov_mle(data[i,])
    ev <- eigen(A)
    lambda <- sort(ev$values,decreasing = TRUE)
    theta_hat <- lambda[1]/sum(lambda)
    return (theta_hat)
}

1. Use bootstrap to estimate the bias and standard error of $\hat{\theta}$.


In [92]:
#handmade
theta.hat <- theta_hat(data)
n <- nrow(data)
B <- 2000 
theta.b <- numeric(B)
for (b in 1:B) {
    i <- sample(1:n, size = n, replace = TRUE)
    theta.b[b] <- theta_hat(data,i)
}
bias <- mean(theta.b - theta.hat)
se <- sd(theta.b)
print(list(est=theta.hat, bias = bias,se = se))

$est
[1] 0.619115

$bias
[1] 0.0002556921

$se
[1] 0.04943359



In [93]:
#using package
obj <- boot(data, theta_hat, R = 2000)
obj


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = data, statistic = theta_hat, R = 2000)


Bootstrap Statistics :
    original       bias    std. error
t1* 0.619115 0.0007689663  0.04769692

2. Obtain the jackknife estimates of bias and standard error of $\hat{\theta}$.


In [94]:
n <- nrow(data)

theta.hat <- theta_hat(data)
theta.jack <- numeric(n)
for (i in 1:n)
    theta.jack[i] <- theta_hat(data[-i, ])
bias <- (n - 1) * (mean(theta.jack) - theta.hat)
#se <- sd(theta.jack)
se <- sqrt((n-1) * mean((theta.jack - mean(theta.jack))^2))
print(list(bias = bias,se = se))

$bias
[1] 0.001069139

$se
[1] 0.04955231



3. Compute 95% percentile and BCa confidence intervals for $\hat{\theta}$.

In [95]:
boot.ci(obj,type=c("perc","bca"))

BOOTSTRAP CONFIDENCE INTERVAL CALCULATIONS
Based on 2000 bootstrap replicates

CALL : 
boot.ci(boot.out = obj, type = c("perc", "bca"))

Intervals : 
Level     Percentile            BCa          
95%   ( 0.5191,  0.7074 )   ( 0.5171,  0.7048 )  
Calculations and Intervals on Original Scale